In [4]:
import pandas as pd
from pandas.api.types import CategoricalDtype  # 用于DataFrame排序用。


def sort_list(series_or_dataframe, list_sort, column_sort=None):
    """
    功能：按照指定列和排序列表对Series或DataFrame排序;
    参数：series_or_dataframe Type Series or DataFrame,需要排序的数据;
         list_sort 排序列表集合，[[]]形式；
         column 需要排序的列
    返回：series，Type Series或dataframe,Type DataFrame
    """
    # 判断series_or_dataframe是否为pd.Series
    if isinstance(series_or_dataframe, pd.Series):
        # 将pd.Series转换成pd.DataFrame
        sort_list_df = pd.DataFrame(series_or_dataframe)
        # 重置索引，并保留索引
        sort_list_df = sort_list_df.reset_index()
        # 将列名改为index,values
        sort_list_df.columns = ['index', 'values']
        # 选出排序列中数据在排序列表中的数据，为了后续不在其中的不变为NAN
        sort_list_df1 = sort_list_df[sort_list_df['index'].isin(list_sort)].copy()
        # 选出排序列表中数据不在排序列表中的数据
        sort_list_df2 = sort_list_df[~sort_list_df['index'].isin(list_sort)].copy()
        # 将index排序列转换为category数据类型
        sort_list_df1['index'] = sort_list_df1['index'].astype('category')
        # 将index列按照list_sort排序
        sort_list_df1['index'].cat.set_categories(list_sort, inplace=True)
        sort_list_df1.sort_values('index', ascending=True, inplace=True)
        # 合并两个数据
        sort_list_df = pd.concat([sort_list_df1, sort_list_df2])
        # 转回Series类型
        series = pd.Series(sort_list_df['values'].values, index=sort_list_df['index'])
        return series
    # 判断series_or_dataframe是否为pd.DataFrame
    elif isinstance(series_or_dataframe, pd.DataFrame):
        # 将排序列中没有在排序列表的数据加入排序列表
        for i in range(len(column_sort)):
            list_sort[i] = list_sort[i] + list(set(series_or_dataframe[column_sort[i]]).difference(set(list_sort[i])))
        
        for i in range(len(column_sort)):
            # 将排序列表转换为CategoricalDtype类型
            cat_order = CategoricalDtype(
                list_sort[i],
                ordered=True
            )
            # 将排序列转换为排序列表类型
            series_or_dataframe[column_sort[i]] = series_or_dataframe[column_sort[i]].astype(cat_order)
        #按照排序列表排序
        dataframe = series_or_dataframe.sort_values(column_sort, axis=0, ascending=[True] * (len(column_sort)))
        return dataframe
if __name__ == "__main__":
    sr=pd.Series([1,2,3,4],index=['A','B','C','D'])
    df=pd.DataFrame(
                    [
                        ['A',1,5,pd.to_datetime('2020-12-31'),pd.to_datetime('2021-1-8')],
                        ['D',1,4,pd.to_datetime('2021-2-26'),pd.to_datetime('2021-3-7')],
                        ['C',3,6,pd.to_datetime('2021-1-20'),pd.to_datetime('2022-3-8')],
                    ]
                    ,columns=['一','二','三','开始日期','结束日期']
                    )
    sort_list1=['A','C','B','D']
    sort_list2=[['A','C','B','D']]
    sl=sort_list(sr,sort_list1)
    sl1=sort_list(df,sort_list2,['一'])
sl1
    

,一,二,三,开始日期,结束日期
0,A,1,5,2020-12-31,2021-01-08
2,C,3,6,2021-01-20,2022-03-08
1,D,1,4,2021-02-26,2021-03-07


In [ ]:
def sfb_plan_piovt_table(df, piovt_index, piovt_columns, piovt_columns_values, piovt_values, all_name=None):
    """
    piovt_index 透视表需要参与分类的列
    piovt_index 透视表需要统计的列
    piovt_values 透视表需要计算的列
    table_columns 透视表重新命名列名
    all_name  All更改名字
    """
    if all_name:
        # piovt_columns中包括的值
        piovt_columns_items = list[set[df[piovt_columns].to_list()]]
        if len(piovt_columns_items) != len(piovt_columns_values):
            piovt_columns_diff = piovt_columns_values.difference(piovt_columns_items)
            for item in range(piovt_columns_diff):
                df.loc[len(df)], df.loc[len(df),piovt_columns]= '过程测试'，item
                
        # 建立基础透视表
        df_table = df.pivot_table(index=piovt_index,
                                  columns=piovt_columns,
                                  values=piovt_values,
                                  aggfunc=len,
                                  fill_value=0
                                  )
        df_table.drop('过程测试',axis=0)
        # 将index变为列
        df_table.reset_index(inplace=True)
        # 标准透视表列名
        df_table_columns_standard = piovt_index + piovt_columns_values
        # 透视表列名
        df_table = df_table[df_table_columns_standard]
      
        for item in piovt_index:
            df_table.loc[len(df_table) - 1, item] = all_name
        # 计算完成率
        df_table['完成率'] = df_table['是'] / df_table['合计']
        # 设置完成率为百分数
        df_table['完成率'] = df_table['完成率'].apply(lambda x: '%.2f%%' % (x * 100))
        df_table_piovt_all = []
        if len(piovt_index) > 1:
            for item in piovt_index:
                if item != piovt_index[-1]:
                    df_table_piovt_index = df.pivot_table(index=item,
                                                          columns=piovt_columns,
                                                          values=piovt_values,
                                                          aggfunc=len,
                                                          fill_value=0,
                                                          margins=True
                                                          )
                    # 将index变为列
                    df_table_piovt_index.reset_index(inplace=True)
                    if len(list(df_table_piovt_index)) < 4:
                        df_table_piovt_index[('业务领域', '否')] = 0
                        df_table_piovt_index = df_table_piovt_index[[(item, ''), ('业务领域', '否'), ('业务领域', '是'), ('业务领域', 'All')]]

                    print(df_table_piovt_index)
                    df_table_piovt_index.columns = [item, '否', '是', '合计']
                    # 将All名称按照给定名字更改
                    df_table_piovt_index.loc[len(df_table_piovt_index) - 1, item] = all_name
                    # 计算完成率
                    df_table_piovt_index['完成率'] = df_table_piovt_index['是'] / df_table_piovt_index['合计']
                    # 设置完成率为百分数
                    df_table_piovt_index['完成率'] = df_table_piovt_index['完成率'].apply(lambda x: '%.2f%%' % (x * 100))
                    df_table_piovt_all.append(df_table_piovt_index)
            df_table_piovt = pd.concat(df_table_piovt_all, ignore_index=True)
            df_table = pd.concat([df_table, df_table_piovt], ignore_index=True)
            df_table = df_table[df_table[piovt_index[-1]] != all_name]
            df_table.reset_index(drop=True, inplace=True)
            for i in range(len(df_table)):
                if isinstance(df_table[piovt_index[-1]][i], float):
                    df_table.loc[i, piovt_index[-1]] = df_table[piovt_index[-2]][i]
    else:
        # 建立基础透视表
        df_table = df.pivot_table(index=piovt_index,
                                  columns=piovt_columns,
                                  values=piovt_values,
                                  aggfunc=len,
                                  fill_value=0,
                                  margins=True
                                  )
        # 将index变为列
        df_table.reset_index(inplace=True)
        if len(list(df_table)) < len(table_columns):
            df_table[('业务领域', '否')] = 0
            if len(table_columns) == 4:
                df_table = df_table[[(table_columns[0], ''), ('业务领域', '否'), ('业务领域', '是'), ('业务领域', 'All')]]
            else:
                df_table = df_table[[(table_columns[0], ''), (table_columns[1], ''), ('业务领域', '否'), ('业务领域', '是'), ('业务领域', 'All')]]
        print(df_table)
        # 将All名称按照给定名字更改
        df_table.columns = table_columns
        # 去除All行
        df_table = df_table.head(len(df_table)-1)
        # 计算完成率
        df_table['完成率'] = df_table['是'] / df_table['合计']
        # 设置完成率为百分数
        df_table['完成率'] = df_table['完成率'].apply(lambda x: '%.2f%%' % (x * 100))

    return df_table

In [21]:
import pandas as pd
b=[('业务领域', '否'),('业务领域', '是'),('业务领域', '是'),('业务领域', '')]
df = pd.DataFrame(b,columns=['慕课','完成度'])
df.loc[len(df)]='辅助'
df.loc[len(df)-1,'完成度']='非'
df.loc[len(df)]='辅助'
df.loc[len(df)-1,'完成度']='非'
x =set(df['完成度'].to_list())
print(df)

     慕课 完成度
0  业务领域   否
1  业务领域   是
2  业务领域   是
3  业务领域    
4    辅助   非
5    辅助   非


In [22]:
b=[('业务领域', '否'),('业务领域', '是'),('业务领域', '是'),('业务领域', '')]
x=[i[1] for i in b ]
print(x)

['否', '是', '是', '']


In [45]:
import pandas as pd
def piovt_table_all_item(df, piovt_index, piovt_columns, piovt_columns_values, piovt_values, all_name=None):
    """
    df:DataFrame，需要透视的表
    piovt_index 透视表需要参与分类的列
    piovt_columns 透视表需要统计的列
    piovt_columns_values 透视表需要统计的列中元素取值
    piovt_values 透视表需要计算的列    
    all_name  统计合计，且合计All更改名字
    """
    
    if all_name:
        # piovt_columns中包括的值
        piovt_columns_items = list(set(df.loc[:,piovt_columns].to_list()))
        if len(piovt_columns_items) != len(piovt_columns_values):
            piovt_columns_diff = list(set(piovt_columns_values).difference(set(piovt_columns_items)))
            for item in piovt_columns_diff:
                df.loc[len(df)]= '过程测试'
                df.loc[len(df)-1,piovt_columns]= item
               
            # 建立基础透视表
            df_table = df.pivot_table(index=piovt_index,
                                      columns=piovt_columns,
                                      values=piovt_values,
                                      aggfunc=len,
                                      fill_value=0
                                      )
            df_table.drop('过程测试',axis=0,inplace=True)
        else:
             # 建立基础透视表
            df_table = df.pivot_table(index=piovt_index,
                                      columns=piovt_columns,
                                      values=piovt_values,
                                      aggfunc=len,
                                      fill_value=0
                                      )
        # 将index变为列
        df_table.reset_index(inplace=True)
        print(df_table)
        # 标准透视表列名
        df_table_columns_standard = piovt_index + piovt_columns_values
        # 透视表列名
        df_table = df_table[df_table_columns_standard]
        print(df_table)
        df_table.columns=piovt_index + piovt_columns_values
        #对行求和
        df_table.loc[len(df_table)] = df_table[piovt_columns_values].apply(lambda x: x.sum())
        df_table.loc[len(df_table)-1,piovt_index] = all_name
        #对列求和
        df_table.loc[:,'合计'] = df_table[piovt_columns_values].apply(lambda x: x.sum(), axis=1)
    else:
         # piovt_columns中包括的值
        piovt_columns_items = list(set(df.loc[:,piovt_columns].to_list()))
        if len(piovt_columns_items) != len(piovt_columns_values):
            piovt_columns_diff = list(set(piovt_columns_values).difference(set(piovt_columns_items)))
            for item in piovt_columns_diff:
                df.loc[len(df)]= '过程测试'
                df.loc[len(df)-1,piovt_columns]= item
               
            # 建立基础透视表
            df_table = df.pivot_table(index=piovt_index,
                                      columns=piovt_columns,
                                      values=piovt_values,
                                      aggfunc=len,
                                      fill_value=0
                                      )
            df_table.drop('过程测试',axis=0,inplace=True)
        else:
             # 建立基础透视表
            df_table = df.pivot_table(index=piovt_index,
                                      columns=piovt_columns,
                                      values=piovt_values,
                                      aggfunc=len,
                                      fill_value=0
                                      )
        # 将index变为列
        df_table.reset_index(inplace=True)
        print(df_table)
        # 标准透视表列名
        df_table_columns_standard = piovt_index + piovt_columns_values
        # 透视表列名
        df_table = df_table[df_table_columns_standard]
        print(df_table)
        df_table.columns=piovt_index + piovt_columns_values
    return df_table
if __name__ == "__main__":
    sr=pd.Series([1,2,3,4],index=['A','B','C','D'])
    df=pd.DataFrame(
                    [
                        ['A','1',5,'孔令',2],
                        ['D','1',4,'李黎',2],
                        ['D','1',6,'李黎',2],
                    ]
                    ,columns=['一','二','三','开始日期','结束日期']
                    )
#     sort_list1=['A','C','B','D']
#     sort_list2=[['A','C','B','D']]
#     sl=sort_list(sr,sort_list1)
#     sl1=sort_list(df,sort_list2,['一'])
    tb=piovt_table_all_item(df, ['一','开始日期','结束日期'], '二', ['是','否','1'], '三')
tb 

二  一 开始日期 结束日期  1  否  是
0  A   孔令    2  1  0  0
1  D   李黎    2  2  0  0
二  一 开始日期 结束日期  是  否  1
0  A   孔令    2  0  0  1
1  D   李黎    2  0  0  2


,一,开始日期,结束日期,是,否,1
0,A,孔令,2,0,0,1
1,D,李黎,2,0,0,2
